In [1]:
from average_traces import AverageTraces
from Session import Session, SessionLite
import matplotlib.pyplot as plt
from matplotlib import gridspec

In [2]:
traces = AverageTraces('dff')

FileNotFoundError: [Errno 2] No such file or directory: '/Users/danieldaylewis/Documents/code/sessions_lite_flu.pkl'

In [ ]:
traces.build_trace_dict()

In [ ]:
plt.figure(figsize=(20, 20))
outer = gridspec.GridSpec(3,1, hspace=0.4, wspace=0.8)
traces.s1s2_plot((-0.02, 0.06), outer[0], stim_type='test', balance=False)